# This code extracts text from PDF tables, by looking at specific pages.

## Install / import necessary libs

In [3]:
# import tabula
from tabula import read_pdf
import PyPDF2
import re
import os
import pdfplumber
import pandas as pd
import sys


## Extract text

### First we look at the files Emergency Food Assistance*.pdf and extract tables. 

### Steps:
### 1. Fetch the page first with the string "ANNEX I-A" - using PyPDF2 lib.
### 2. Extract the table using tabula-py lib. 

In [56]:
# Convenience function that takes a filename and provides a list of pagenumber
def get_table_from_pdf(filename):
    object = PyPDF2.PdfFileReader(filename)

    # get number of pages
    NumPages = object.getNumPages()
#    print(NumPages)

    # define keyterms
    searchString = "Best track"
    #   if(filename.startswith('Emergency Food Assistance')):
    #     searchString = "ANNEX I-A"

    # extract text and do the search
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        textPDF = PageObj.extractText() 
#         ResSearch = re.search(searchString, textPDF, flags = 1)
#         if ResSearch:
#        print("For file: ", filename, ", the page number is: ", str(i))
        col1 = []
        try:
            df = read_pdf(filename, pages = i)
            for col in df[0].columns: 
                col1.append(col)
#                print(col1)
            if(col1[0].startswith('Date/Time')):
                return df[0]
        except:
            None
        #Be careful as this page number needs to be incremented by 1 in the calling code 

In [57]:
folder = '/home/dev/course-v3/nbs/dl1/omdena1/pdf_from_links/1995_pdf/'
# file = 'Emergency Food Assistance to Victims of Hurricane Irma in Cuba.pdf'
file1 = 'EP212019_Twenty_One-E.pdf'
file2 = 'AL101999_Harvey.pdf'
file3 = 'EP192000_Rosa.pdf'
file4 = 'AL012000_One.pdf'
file5 = 'AL082005_Harvey.pdf'
file6 = 'AL102015_Ida.pdf'
# get_table_from_pdf(folder + file4)
#Assistance to Victims of Hurricane Irma and Hurricane Jose in the Eastern_Caribbean.pdf
#Emergency Food Assistance to Communities Affected by_Hurricane Maria in Dominica.pdf

In [58]:
df_tab = get_table_from_pdf(folder + file6)
df_tab

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 3:05:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:05:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:05:28 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Date/Time,Latitude,Longitude,Pressure,Unnamed: 0,Unnamed: 1
0,NaN,NaN,NaN,NaN,Wind,NaN
1,(UTC),(°N),(°W),(mb),NaN,NaN
2,NaN,NaN,NaN,NaN,Speed,Stage
3,NaN,NaN,NaN,NaN,(kt),NaN
4,15 / 1200,9.4,24.2,1010,25,low
5,15 / 1800,9.7,25.5,1010,25,""""
6,16 / 0000,10.0,26.6,1010,25,""""
7,16 / 0600,10.4,27.5,1010,25,""""
8,16 / 1200,10.7,28.4,1010,25,""""
9,16 / 1800,11.0,29.4,1009,25,""""


In [201]:
df_tab[['1','2','3', 'Latitude']] = df_tab['Date/Time Latitude'].str.split(" ",expand=True,)
# df_tab


# cols = df_tab.columns.tolist()
# print(cols)

In [202]:
df_tab['Date/Time Latitude'] = df_tab['1'] + df_tab['2'] + df_tab['3']
# df_tab

In [203]:
df_tab = df_tab.drop(['1','2','3'], axis=1)
# df_tab

In [204]:
cols = df_tab.columns.to_list()
# cols

In [59]:
cols = ['Date/Time Latitude',
        'Latitude',
         'Longitude',
         'Pressure',
         'Wind Speed',
         'Unnamed: 0']
df_tab = df_tab[cols]
# df_tab

KeyError: "['Date/Time Latitude', 'Wind Speed'] not in index"

In [60]:
df_tab.rename(columns={df_tab.columns[0]:'Date/Time',
                       df_tab.columns[1]:'Latitude',
                       df_tab.columns[2]:'Longitude',
                       df_tab.columns[3]:'Pressure',
                       df_tab.columns[4]:'Wind Speed'}, 
              inplace=True)

In [61]:
df_tab

,Date/Time,Latitude,Longitude,Pressure,Wind Speed,Unnamed: 1
0,NaN,NaN,NaN,NaN,Wind,NaN
1,(UTC),(°N),(°W),(mb),NaN,NaN
2,NaN,NaN,NaN,NaN,Speed,Stage
3,NaN,NaN,NaN,NaN,(kt),NaN
4,15 / 1200,9.4,24.2,1010,25,low
5,15 / 1800,9.7,25.5,1010,25,""""
6,16 / 0000,10.0,26.6,1010,25,""""
7,16 / 0600,10.4,27.5,1010,25,""""
8,16 / 1200,10.7,28.4,1010,25,""""
9,16 / 1800,11.0,29.4,1009,25,""""


In [62]:
df_tab = df_tab.fillna(0)
df_tab

,Date/Time,Latitude,Longitude,Pressure,Wind Speed,Unnamed: 1
0,0,0,0,0,Wind,0
1,(UTC),(°N),(°W),(mb),0,0
2,0,0,0,0,Speed,Stage
3,0,0,0,0,(kt),0
4,15 / 1200,9.4,24.2,1010,25,low
5,15 / 1800,9.7,25.5,1010,25,""""
6,16 / 0000,10.0,26.6,1010,25,""""
7,16 / 0600,10.4,27.5,1010,25,""""
8,16 / 1200,10.7,28.4,1010,25,""""
9,16 / 1800,11.0,29.4,1009,25,""""


In [42]:
flag = True
try:
    flag = True
    print('1')
    index = df_tab[df_tab['Date/Time'].str.contains('UTC', na=False)].index.values.tolist()
    print('2')
    #index[0]
except:
    flag = False
    
flag

1
2


True

In [41]:
df_tab[df_tab['Date/Time'].str.contains('UTC')]

ValueError: cannot index with vector containing NA / NaN values

In [224]:
if(flag):
    df_tab_mod1 = df_tab_mod.drop(index[0])

In [225]:
df_tab_mod1

,Date/Time,Latitude,Longitude,Pressure,Wind Speed,Unnamed: 0
1,02/1800,28.2,68.8,1006,30,tropical depression
2,03/0000,28.9,68.7,1005,30,""""
3,03/0600,29.5,68.6,1003,40,tropical storm
4,03/1200,30.3,68.3,1001,45,""""
5,03/1800,30.9,67.7,999,50,""""
6,04/0000,31.3,66.4,997,50,""""
7,04/0600,31.6,65.0,995,50,""""
8,04/1200,31.9,63.6,995,50,""""
9,04/1800,32.0,62.1,994,55,""""
10,05/0000,32.1,60.5,994,55,""""


In [212]:
df_tab_mod1['Wind Speed'] = df_tab_mod1['Wind Speed'].astype(int)
df_tab_mod1['Pressure'] = df_tab_mod1['Pressure'].astype(int)

In [213]:
df_tab_mod1.max()

Date/Time            12/1200
Latitude                44.5
Longitude               68.8
Pressure                1009
Wind Speed                55
Unnamed: 0    tropical storm
dtype: object

In [214]:
d = df_tab_mod1[df_tab_mod1['Wind Speed'] == df_tab_mod1['Wind Speed'].max()].iloc[0]
d

Date/Time     04/1800
Latitude         32.0
Longitude        62.1
Pressure          994
Wind Speed         55
Unnamed: 0          "
Name: 9, dtype: object

In [63]:
folder = '/home/dev/course-v3/nbs/dl1/omdena1/pdf_from_links/1995_pdf/'
test_folder = '/home/dev/course-v3/nbs/dl1/omdena1/pdf_from_links/test/'

df_cyclone_features = pd.DataFrame(columns=['Filename', 
                                            'UTC_Date_Time', 
                                            'Latitude', 
                                            'Longitude', 
                                            'Wind_Speed', 
                                            'Pressure'])

error_log = []

i = 0
for file in os.listdir(folder):
     if(i < 1000):
        try:
            #print(file)
            i = i + 1
            df_original = get_table_from_pdf(folder + file)
            #print(df_original.head())

            #Get column names
            cols = df_original.columns.to_list()
            #print(cols[0])
            #print('here1')
            if('Latitude' in cols[0]):
                #print('here2')
                df_original[['1','2','3', 'Latitude']] = df_original['Date/Time Latitude'].str.split(" ",expand=True)
                df_original['Date/Time Latitude'] = df_original['1'] + df_original['2'] + df_original['3']
                df_original = df_original.drop(['1','2','3'], axis=1)

                cols1 = ['Date/Time Latitude',
                        'Latitude',
                         'Longitude',
                         'Pressure',
                         'Wind Speed',
                         'Unnamed: 0']
                df_original = df_original[cols1]

            #print('here3')
            #Change column names as some pdf have Wind and othes have Wind Speed
            df_original.rename(columns={df_original.columns[0]:'Date/Time',
                       df_original.columns[1]:'Latitude',
                       df_original.columns[2]:'Longitude',
                       df_original.columns[3]:'Pressure',
                       df_original.columns[4]:'Wind Speed'}, 
                      inplace=True)
            #print('1')
            #print(df_original.head())
            #Retrieve index of row that has (UTC) in a row
            flag = True
            try:
                #print('2')
                index = df_original[df_original['Date/Time'].str.contains('UTC', na=False)].index.values.tolist()
                if(index == []): flag = False
                #print(index)
                #print('3')
            except:
                #print('4')
                flag = False

            df_fillna = df_original.fillna(0)

            if(flag):
                df_mod = df_fillna.drop(index[0])
            else:
                df_mod = df_fillna

            df_mod['Wind Speed'] = df_mod['Wind Speed'].astype(int)
            df_mod['Pressure'] = df_mod['Pressure'].astype(int)

            final_values = df_mod[df_mod['Wind Speed'] == df_mod['Wind Speed'].max()].iloc[0]

            utc_date_time = final_values[0]
            latitude = final_values[1]
            longitude = final_values[2]
            pressure = final_values[3]
            windspeed = final_values[4]


            df_cyclone_features = df_cyclone_features.append({'Filename': file, 
                                                  'UTC_Date_Time': utc_date_time, 
                                                  'Latitude': latitude,
                                                  'Longitude': longitude,
                                                  'Wind_Speed': windspeed,
                                                  'Pressure': pressure},
                                                  ignore_index=True)
                        
        except:
            error_log.append(file)
            #print(sys.exc_info())
            #print(f'Filename that had a problem {file}')
        

        

'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 3:09:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:14 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:09:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:15 PM org.apache.pdfbox.pdmodel.font.PD

Got stderr: Mar 26, 2020 3:09:37 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:37 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:09:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:09:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:09:41 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:09:41 PM o

Got stderr: Mar 26, 2020 3:10:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:10:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:10:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:10:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 3:10:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:10:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:10:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:10:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:10:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:10:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:10:13 PM org.a

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 3:12:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:16 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:12:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:18 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:18 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:12:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:12:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:20 PM org

'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 3:12:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:12:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:13:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:13:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:13:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:13:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 3:13:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:13:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:13:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 3:13:02 PM org.apache.pdfbox.pd

...

Got stderr: Mar 26, 2020 5:21:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:21:58 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:21:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:21:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:21:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:21:59 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 5:22:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:22:00 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:22:01 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 5:22:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:22:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:22:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 5:22:04 PM o

Got stderr: Mar 26, 2020 5:22:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial-BoldMT are not implemented in PDFBox and will be ignored
Mar 26, 2020 5:22:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ArialMT are not implemented in PDFBox and will be ignored
Mar 26, 2020 5:22:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial-BoldMT are not implemented in PDFBox and will be ignored
Mar 26, 2020 5:22:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ArialMT are not implemented in PDFBox and will be ignored
Mar 26, 2020 5:22:19 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial-BoldMT are not implemented in PDFBox and will be ignored
Mar 26, 2020 5:22:19 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenT

'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 5:23:44 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:23:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:23:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:23:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:23:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:23:45 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 5:23:46 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:23:46 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:23:46 PM org.apache.pdfbox.pdmodel.font.PDTr

Got stderr: Mar 26, 2020 5:24:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:12 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 5:24:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:20 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:21 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 26, 2020 5:24:22 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:22 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:22 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:22 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:24:22 PM org.apache.pdfbox.

Got stderr: Mar 26, 2020 5:25:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 5:25:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 26, 2020 5:25:40 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Mar 26, 2020 5:25:41 PM org.apache.pdf

In [71]:
df_error = pd.DataFrame(data={"Filename_error": error_log})
df_error.to_csv("/home/dev/course-v3/nbs/dl1/omdena1/pdf_from_links/error_file.csv", sep=',',index=False)
# error_log

In [67]:
df_cyclone_features

,Filename,UTC_Date_Time,Latitude,Longitude,Wind_Speed,Pressure
0,EP012014_Amanda.pdf,25 / 1200,11.7,111.1,135,932
1,EP062017_Fernanda.pdf,15 / 0000,10.7,120.3,125,948
2,EP022009_Andres.pdf,23 / 0600,17.2,103.1,70,984
3,EP172000_Olivia.pdf,03 / 1800,15.4,104.5,55,994
4,AL172007_Olga.pdf,11 / 1800,18.5,68.4,50,1003
...,...,...,...,...,...,...
564,EP052001_Dalila.pdf,24 / 1200,18.2,109,65,987
565,AL102011_Ten.pdf,25 / 0000,12.3,29.0,30,1006
566,EP102011_Jova.pdf,10 / 1800,16.5,106.5,110,955
567,AL082012_Gordon.pdf,19 / 0000,34.3,35.0,95,965


In [72]:
df_cyclone_features.to_csv('/home/dev/course-v3/nbs/dl1/omdena1/pdf_from_links/Output_from_pdf.csv')